In [46]:
import requests
import json
import math

def crear_rambla(id, dateLastValueReported, description, locality, postalCode, source, location, temperature, 
                 providedByTemp, obsTemp, precipitation, providedByPrec, obsPrec, humidity, providedByHum, 
                 obsHum, atmosphericPressure, providedByAtPress, obsAtPress, windDirection, providedByWindDir, 
                 obsWindDir, windSpeed, providedBywindSp, obsWindSp , waterFlowValues, waterLevelValues, 
                 section, nextSection, previousSection, name, measurementsPlaces, volume, providedByVol, obsVol, 
                 pressure, providedByPress, obsPress, conductivity, providedByCond, obsCond):
    rambla_content = {
        "id": f"urn:ngsi-ld:Ravine:{id}",
        "identifier":{
            "type": "Property",
            "value": f"urn:ngsi-ld:Ravine:{id}"
        },
        "type": "Ravine",
        "dateLastValueReported": {
            "type": "Property",
            "value": dateLastValueReported
        },
        "description": {
            "type": "Property",
            "value": description
        },
        "address": {
            "type": "Property",
            "value": {
                "addressCountry": "ES",
                "addressRegion": "Murcia",
                "addressLocality": locality,
                "postalCode": postalCode,
            },
            "verified": {
                "type": "Property",
                "value": True
            }
        },
        "source": {
            "type": "Property",
            "value": source
        },
        "location": {
            "type": "GeoProperty",
            "value": {
                "type": "Point",
                "coordinates": location
            }
        },
        "name": {
            "type": "Property",
            "value": name
        },
        "@context": "https://raw.githubusercontent.com/mariete1223/MarMenor/main/data_models_description/datamodels.context-ngsi.jsonld"
    }

    # Función auxiliar para agregar una propiedad si no es None
    def add_property_if_not_none(property_name, value, observed_at, unit_code, provided_by):

        singleValues = ["temperature", "precipitation", "humidity"]
        
            
        if isinstance(value, str):
            rambla_content[property_name] = {
                "type": "Relationship",
                "object": value
            }
        elif value is not None and property_name in singleValues and not math.isnan(value) :
            rambla_content[property_name] = {
                "type": "Property",
                "value": value,
                "observedAt": observed_at,
                "unitCode": unit_code,
                "providedBy": {
                    "type": "Relationship",
                    "object": provided_by
                }
            }
            if (property_name in measurementsPlaces) and  (measurementsPlaces[property_name] is not None):
                rambla_content[property_name]["measurementPlace"] = {
                    "value": measurementsPlaces[property_name]
                }
        elif value is not None:
            rambla_content[property_name] = {
                "value": value
            }

            
        

    # Agregar propiedades si no son None
    add_property_if_not_none("temperature", temperature, obsTemp, "CEL", providedByTemp)
    add_property_if_not_none("pressure", pressure, obsPress, "bar", providedByPress)
    add_property_if_not_none("volume", volume, obsVol, "m3", providedByVol)
    add_property_if_not_none("conductivity", conductivity, obsCond, "CEL", providedByCond)
    add_property_if_not_none("windSpeed", windSpeed, obsWindSp, "m/s", providedBywindSp)
    add_property_if_not_none("windDirection", windDirection, obsWindDir, "°", providedByWindDir)
    add_property_if_not_none("atmosphericPressure", atmosphericPressure, obsAtPress, "CEL", providedByAtPress)
    add_property_if_not_none("precipitation", precipitation, obsPrec, "mm", providedByPrec)
    add_property_if_not_none("humidity", humidity, obsHum, "%", providedByHum)
    add_property_if_not_none("waterFlowValues", waterFlowValues, None, "m3/s", None)
    add_property_if_not_none("waterLevelValues", waterLevelValues, None, "m", None)
    add_property_if_not_none("nextSection", nextSection, None, "%", None)
    add_property_if_not_none("previousSection", previousSection, None, "m3/s", None)
    add_property_if_not_none("section", section, None, "m3/s", None)

    url = "http://localhost:1026/ngsi-ld/v1/entities/"

    headers = {
        "Content-Type": "application/ld+json"
    }

    response = requests.post(url, headers=headers, json=rambla_content)
    print(rambla_content)
    print(response.status_code, response.text)


In [2]:
def crear_sensor(id, sensor_type, locality, postalCode, location, controlledProperty, measuredProperty, measureValue , unitCode, source, controlledAsset, dateLastValueReported, measureType):
    sensor_content = {
        "id": f"urn:ngsi-ld:{sensor_type}:{id}",
        "identifier":{
            "type": "Property",
            "value": f"urn:ngsi-ld:{sensor_type}:{id}"
        },
        "type": sensor_type,
        "category": {
            "type": "Property",
            "value": "sensor"
        },
        "address": {
            "type": "Property",
            "value": {
                "addressCountry": "ES",
                "addressRegion": "Murcia",
                "addressLocality": locality,
                "postalCode": postalCode
            },
            "verified": {
                "type": "Property",
                "value": True
            }
        },
        "location": {
            "type": "GeoProperty",
            "value": {
                "type": "Point",
                "coordinates": location
            }
        },
        "controlledProperty": {
            "type": "Property",
            "value": controlledProperty
        },
        measuredProperty: {
            "type": "Property",
            "value": measureValue,
            "observedAt": dateLastValueReported,
            "unitCode": unitCode
        },
        "measureType": {
            "type": "Property",
            "value": measureType
        },
        "source": {
            "type": "Property",
            "value": source
        },
        "controlledAsset": {
            "type": "Relationship",
            "object": controlledAsset
        },
        "@context": "https://raw.githubusercontent.com/mariete1223/MarMenor/main/data_models_description/datamodels.context-ngsi.jsonld"
    }
    
    if unitCode is None:
        del sensor_content[measuredProperty]["unitCode"]
        
    if measureType == "multipleValue":
        del sensor_content[measuredProperty]
        if (measureValue is not None):
            sensor_content[measuredProperty+"Values"] =[measureValue]
        else:
            sensor_content[measuredProperty+"Values"] = [[{"depth": {"value": -99, "unitCode": "MTR"}, "value": {"value": -99, "unitCode": "CEL"}, "observedAt": dateLastValueReported}]]
    

    url = "http://localhost:1026/ngsi-ld/v1/entities/"

    headers = {
        "Content-Type": "application/ld+json"
    }

    response = requests.post(url, headers=headers, json=sensor_content)
    if(response.status_code == 409):
        print(f"urn:ngsi-ld:{sensor_type}:{id}")
    

In [3]:
import math

def crear_piezometro(id, locality, postalCode, location, description, source, dateLastValueReported, numberInNetwork,
                      temperature, obsTemp, providedByTemp, conductivity, obsCond, providedByCond,
                      piezometricLevel, obsPiez, providedByPiez, salinity, obsSal, providedBySal,
                      tds, obsTDS, providedByTDS, name):

    piezometer_content = {
        "id": f"urn:ngsi-ld:PiezometricPoint:{id}",
        "identifier":{
            "type": "Property",
            "value": f"urn:ngsi-ld:PiezometricPoint:{id}"
        },
        "type": "PiezometricPoint",
        "category": {
            "type": "Property",
            "value": "sensor"
        },
        "address": {
            "type": "Property",
            "value": {
                "addressCountry": "ES",
                "addressRegion": "Murcia",
                "addressLocality": locality,
                "postalCode": postalCode
            },
            "verified": {
                "type": "Property",
                "value": True
            }
        },
        "location": {
            "type": "GeoProperty",
            "value": {
                "type": "Point",
                "coordinates": location
            }
        },
        "description": {
            "type": "Property",
            "value": description
        },
        "source": {
            "type": "Property",
            "value": source
        },
        "dateLastValueReported": {
            "type": "Property",
            "value": dateLastValueReported
        },
        "numberInNetwork": {
            "type": "Property",
            "value": 1
        },
         "name": {
            "type": "Property",
            "value": name
        },
        "@context": "https://raw.githubusercontent.com/mariete1223/MarMenor/main/data_models_description/datamodels.context-ngsi.jsonld"
    }

    # Función auxiliar para agregar una propiedad si no es None
    def add_property_if_not_none(property_name, value, observed_at, unit_code, provided_by):
        if value is not None and not math.isnan(value) :
            piezometer_content[property_name] = {
                "type": "Property",
                "value": value,
                "observedAt": observed_at,
                "unitCode": unit_code,
                "providedBy": {
                    "type": "Relationship",
                    "object": provided_by
                }
            }

    # Agregar propiedades si no son None
    add_property_if_not_none("temperature", temperature, obsTemp, "CEL", providedByTemp)
    add_property_if_not_none("conductivity", conductivity, obsCond, "µS/cm", providedByCond)
    add_property_if_not_none("piezometricLevel", piezometricLevel, obsPiez, "msnm", providedByPiez)
    add_property_if_not_none("salinity", salinity, obsSal, "PSU", providedBySal)
    add_property_if_not_none("tds", tds, obsTDS, "mg/L", providedByTDS)

    url = "http://localhost:1026/ngsi-ld/v1/entities/"

    headers = {
        "Content-Type": "application/ld+json"
    }


    response = requests.post(url, headers=headers, json=piezometer_content)
    print(response)



In [4]:
import requests
import json

def crear_boya(id, dateLastValueReported, description, postalCode, locality, location, name, source, temperatureValues=None, chlorophyllValues=None, conductivityValues=None, organicMatterValues=None, oxygenValues=None, pHValues=None, polyethyleneValues=None, salinityValues=None, transparencyValues=None, turbidityValues=None, closeMeasurements=None):
    buoy_content = {
        "id": f"urn:ngsi-ld:Buoy:{id}",
        "identifier":{
            "type": "Property",
            "value":  f"urn:ngsi-ld:Buoy:{id}"
        },
        "type": "Buoy",
        "dateLastValueReported": {
            "type": "Property",
            "value": dateLastValueReported
        },
        "description": {
            "type": "Property",
            "value": description
        },
        "address": {
            "type": "Property",
            "value": {
                "addressCountry": "ES",
                "addressRegion": "Murcia",
                "addressLocality": locality,
                "postalCode": postalCode
            },
        },
        "location": {
            "type": "GeoProperty",
            "value": {
                "type": "Point",
                "coordinates": location
            }
        },
        "name": {
            "type": "Property",
            "value": name
        },
        "source": {
            "type": "Property",
            "value": source
        },
        "temperatureValues": {
            "type": "Property",
            "value": [temperatureValues]},
        "chlorophyllValues": {
            "type": "Property",
            "value":[chlorophyllValues]},
        "conductivityValues": {
            "type": "Property",
            "value":[conductivityValues]},
        "organicMatterValues": {
            "type": "Property",
            "value":[organicMatterValues]},
        "oxygenValues": {
            "type": "Property",
            "value":[oxygenValues]},
        "pHValues": {
            "type": "Property",
            "value":[pHValues]},
        "polyethyleneValues": {
            "type": "Property",
            "value":[polyethyleneValues]},
        "salinityValues": {
            "type": "Property",
            "value":[salinityValues]},
        "transparencyValues": {
            "type": "Property",
            "value":[transparencyValues]
        },
        "turbidityValues": {
            "type": "Property",
            "value":turbidityValues
        },
        "closeMeasurements": closeMeasurements,

        "@context": "https://raw.githubusercontent.com/mariete1223/MarMenor/main/data_models_description/datamodels.context-ngsi.jsonld"
    }
    
    if temperatureValues is None:
        del buoy_content["temperatureValues"]
    if chlorophyllValues is None:
        del buoy_content["chlorophyllValues"]
    if conductivityValues is None:
        del buoy_content["conductivityValues"]
    if organicMatterValues is None:
        del buoy_content["organicMatterValues"]
    if oxygenValues is None:
        del buoy_content["oxygenValues"]
    if pHValues is None:
        del buoy_content["pHValues"]
    if polyethyleneValues is None:
        del buoy_content["polyethyleneValues"]
    if salinityValues is None:
        del buoy_content["salinityValues"]
    if transparencyValues is None:
        del buoy_content["transparencyValues"]
    if turbidityValues is None:
        del buoy_content["turbidityValues"]
    if closeMeasurements is None:
        del buoy_content["closeMeasurements"]

    url = "http://localhost:1026/ngsi-ld/v1/entities/"

    headers = {
        "Content-Type": "application/ld+json"
    }

    try:
        response = requests.post(url, headers=headers, json=buoy_content)
        response.raise_for_status()  # Lanza una excepción para códigos de estado HTTP no exitosos
    except requests.exceptions.RequestException as e:
        print(f"Error creating entity: {e}")

In [5]:
def crear_subscription(description, entity_type, watched_attributes, sent_attributes, query, url):
    subscription_content = {
        "description": description,
        "type": "Subscription",
        "entities": [{"type": entity_type}],
        "watchedAttributes": watched_attributes,
        "notification": {
            "attributes": sent_attributes,
            "format": "normalized",
            "endpoint": {
                "uri": url,
                "accept":  "application/json"
            }
        },
        "@context": "https://raw.githubusercontent.com/mariete1223/MarMenor/main/data_models_description/datamodels.context-ngsi.jsonld"
    }

    def add_property_if_not_none(property_name, query):
        if query is not None:
            subscription_content[property_name] = query

    # Agregar propiedades si no son None
    add_property_if_not_none("q", query)

    url = "http://localhost:1026/ngsi-ld/v1/subscriptions/"

    headers = {
        "Content-Type": "application/ld+json"
    }

    print(subscription_content)
    response = requests.post(url, headers=headers, json=subscription_content)
    if(response.status_code == 409):
        print(description)


In [30]:
def modify_entity( value, observedAt, url, multipleValue = None, place = None):

    headers = {
        'Content-Type': 'application/json',
        'Link': '<https://raw.githubusercontent.com/mariete1223/MarMenor/main/data_models_description/datamodels.context-ngsi.jsonld>; rel="http://www.w3.org/ns/json-ld#context"; type="application/ld+json"'
    }

    if multipleValue is not None:
        modify_content = {
        "value": [multipleValue]
        }
    elif observedAt is not None and place is not None:
        modify_content = {
        "value": value,
        "observedAt": observedAt,
        "measurementPlace": {
            "value": place
            }
        }
    elif observedAt is not None:
        modify_content = {
        "value": value,
        "observedAt": observedAt
        }
    elif place is not None:
        modify_content = {
        "value": value,
        "measurementPlace": {
            "value": place
            }
        }
    else:       
        modify_content = {
            "value": value,
        }

    response = requests.patch(url, headers=headers, json=modify_content)
    print(response.status_code, response.text)
    try:
        if response.status_code != 204:
            print(response.text)
    finally:
        response.close()

In [7]:
def create_attribute(newAttributeName, value, observedAt, url, multipleValue = None):

    headers = {
        'Content-Type': 'application/json',
        'Link': '<https://raw.githubusercontent.com/mariete1223/MarMenor/main/data_models_description/datamodels.context-ngsi.jsonld>; rel="http://www.w3.org/ns/json-ld#context"; type="application/ld+json"'
    }

    if multipleValue is not None:
        modify_content = {
            newAttributeName: {
                 "value": [multipleValue]
            }
        }
    elif observedAt is None:
        modify_content = {
            newAttributeName: {
                "value": value
            }
        }
    else:       
        modify_content = {
            newAttributeName: {
                "value": value,
                "observedAt": observedAt
            }
        }

    response = requests.post(url, headers=headers, json=modify_content)
    print(response.status_code, response.text)
    try:
        if response.status_code != 204:
            print(response.text)
    finally:
        response.close()

In [71]:
def search_close_entities(coordinate_lat,coordinate_long, id_entity):
    url = 'http://localhost:1026/ngsi-ld/v1/entities'
    headers = {
        'Accept': 'application/json',
        'Link': '<https://raw.githubusercontent.com/mariete1223/MarMenor/main/data_models_description/datamodels.context-ngsi.jsonld>; rel="http://www.w3.org/ns/json-ld#context"; type="application/ld+json"'
    }

    params = {
        'geometry': 'Point',
        'coordinates': f'[{coordinate_lat},{coordinate_long}]',
        'georel': 'near;maxDistance==4000',
        'options': 'keyValues',
    }

    response = requests.get(url, headers=headers, params=params)

    data = response.json()
    filtered_data = [entity.get("id",) for entity in data if ('Sensor' not in entity.get('id', '')) and (id_entity not in entity.get('id', '') )]
    return filtered_data 
    

In [72]:
entities_id = {}

In [73]:
import os
import glob
import pandas as pd

# Especifica la ruta de la carpeta principal
carpeta = "C:\\Users\\marie\\OneDrive\\Escritorio\\Antiguo ordenador\\Universidad\\Master 1º\\TFM\\NGSI-LD\\datos\\SAIHdatasActualizados\\piezometros\\"

# Utiliza os.listdir() para obtener una lista de elementos en la carpeta
elementos_en_carpeta = os.listdir(carpeta)

# Filtra solo las carpetas en la lista de elementos
carpetas = [elemento for elemento in elementos_en_carpeta if os.path.isdir(os.path.join(carpeta, elemento))]

# DataFrame vacío para almacenar los datos
dfs = {}


# Imprime la lista de carpetas
for carpeta_actual in carpetas:
    df_total = pd.DataFrame()
    archivos_csv = glob.glob(os.path.join(carpeta+carpeta_actual, "*.csv"))
    for archivo in archivos_csv:
    
        df_temporal = pd.read_csv(archivo)
        if(len(df_total) == 0):
            df_total = df_temporal
        else:
            #Hacer un merge de df_total y df_temporal por la columna "Date"
            df_total = pd.merge(df_total, df_temporal, on="Date", how="inner")
    dfs[carpeta_actual] = df_total

for key, df in dfs.items():
    # Convierte la columna "Date" a tipo datetime
    df["Date"] = pd.to_datetime(df["Date"])
    # Formatea la columna "Date" en el nuevo formato
    df["Date"] = df["Date"].dt.strftime("%Y-%m-%dT%H:%M:%SZ")


In [74]:
diccionario_info_piezometros = {
    "06Z01": {
        "locality": "Calle de Murillo, Los Belones, Cartagena",
        "postalCode": "30385",
        "location": [37.630082, -0.777017],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z01E10",
    },
    "06Z02": {
        "locality": "F-34, Cartagena",
        "postalCode": "30383",
        "location": [37.642663, -0.777275],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z02E10",
    },
    "06Z03": {
        "locality": "Camino Rural XVIII, El Algar, Cartagena",
        "postalCode": "30366",
        "location": [37.654261, -0.848734],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z03E10",
    },
    "06Z05": {
        "locality": "N-332, Cartagena",
        "postalCode": "30368",
        "location": [37.6841, -0.8692],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z05E10",
    },
    "06Z06": {
        "locality": "N-332, Cartagena",
        "postalCode": "30368",
        "location": [ 37.7058, -0.8633],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z06E10",
    },
    "06Z07": {
        "locality": "Camino Rural XIII, El Algar, Cartagena",
        "postalCode": "30395",
        "location": [37.7104, -0.8879],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z07E10",
    },
    "06Z08": {
        "locality": "F-35, Cartagena",
        "postalCode": "30710",
        "location": [37.7511, -0.8828],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z08E10",
    },
    "06Z09": {
        "locality": "Autopista del Mediterraneo, Los Alcazares, Cartagena",
        "postalCode": "30710",
        "location": [37.7452, -0.8679],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z09E10",
    },
    "06Z10": {
        "locality": "Cartagena",
        "postalCode": "30710",
        "location": [37.7618, -0.8673],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z10E10",
    },
    "06Z11": {
        "locality": "Avenida de Muñoz Zambudio, Los Alcazares, Cartagena",
        "postalCode": "30710",
        "location": [37.7543, -0.8588],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z11E10",
    },
    "06Z12": {
        "locality": "Calle Bergantin, San Javier, Cartagena",
        "postalCode": "30739",
        "location": [37.7755, -0.8514],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z12E10",
    },
    "06Z13": {
        "locality": "Calle de Poseidon, Los Alcazares, Cartagena",
        "postalCode": "30710",
        "location": [37.7696, -0.8385],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z13E10",
    },
    "06Z14": {
        "locality": "N-332, San Javier, Cartagena",
        "postalCode": "30730",
        "location": [37.7883, -0.8387],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z14E10",
    },
    "06Z15": {
        "locality": "Cartagena",
        "postalCode": "30730",
        "location": [37.7802, -0.8302],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z15E10",
    },
    "06Z16": {
        "locality": "Calle del Embalse de la Toba, San Javier, Cartagena",
        "postalCode": "30730",
        "location": [37.8181, -0.8286],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z16E10",
    },
    "06Z17": {
        "locality": "Avenida de la Romeria de San Blas 2, San Javier, Cartagena",
        "postalCode": "30720",
        "location": [37.8059, -0.8168],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z17E10",
    },
    "06Z18": {
        "locality": "Autopista del Mediterraneo, San Javier, Cartagena",
        "postalCode": "30730",
        "location": [37.8247, -0.8165],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z18E10",
    },
    "06Z19": {
        "locality": "Calle de Navacerrada 33, San Javier, Cartagena",
        "postalCode": "30720",
        "location": [37.8178, -0.8085],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z19E10",
    },
    "06Z20": {
        "locality": "Calle Francesco Borromini 3, San Pedro del Pinatar, Cartagena",
        "postalCode": "30740",
        "location": [37.8350, -0.8023],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z20E10",
    },
}

In [75]:
import math
import time

def check_exists(place_to_search, value_to_search, array=False):
    if array:
        return place_to_search[value_to_search][0] if value_to_search in place_to_search else None

    return place_to_search[value_to_search] if value_to_search in place_to_search else None

maper_property_sensor = { "Temperatura": "TemperatureSensor", "Conductividad": "ConductivitySensor", "Piezometrico":"PiezometricLevelSensor", "Salinidad":"SalinitySensor",
       "Total solido disuelto (TDS)": "TdsSensor"
}

maper_property_controlled = { "Temperatura": "temperature", "Conductividad": "conductivity", "Piezometrico":"piezometricLevel", "Salinidad":"salinity",
       "Total solido disuelto (TDS)": "tds"
}

maper_property_unitcode = { "Temperatura": "CEL", "Conductividad": "microseconds/cm", "Piezometrico":"msnm", "Salinidad":"PSU",
       "Total solido disuelto (TDS)": "mg/l"
}

link_piezometro_sensores = {}

num_piezometros = 0


counter_connections = 0
counters = {}
for carpeta, valorCarpeta in dfs.items():
    sensores = valorCarpeta.columns
    for i in sensores:
        if counters.get(i) is None:
            counters[i] = 0
    id_entity = f"urn:ngsi-ld:PiezometricPoint:{str(num_piezometros).zfill(3)}"
    number_entity = str(num_piezometros).zfill(3)
    id_piezometro = carpeta.split("-")[0]
    for index, row in valorCarpeta.iterrows():
        counter_connections = counter_connections + 1
        if index == 0:
        
            for contador,valorSensor in enumerate(sensores):
                if valorSensor == "Date":
                    continue
            
                id_sensor = f"urn:ngsi-ld:{maper_property_sensor[sensores[contador]]}:{str(counters[valorSensor]).zfill(3)}"
                if not math.isnan(row[valorSensor]):
                    crear_sensor(
                        id=str(counters[valorSensor]).zfill(3),
                        sensor_type=maper_property_sensor[sensores[contador]],
                        locality=diccionario_info_piezometros[id_piezometro]["locality"],
                        postalCode=diccionario_info_piezometros[id_piezometro]["postalCode"],
                        location=diccionario_info_piezometros[id_piezometro]["location"],
                        measureType="singleValue",
                        controlledProperty=maper_property_controlled[valorSensor],
                        measuredProperty=maper_property_controlled[valorSensor],
                        measureValue=row[valorSensor],
                        unitCode=maper_property_unitcode[valorSensor],
                        source=diccionario_info_piezometros[id_piezometro]["source"],
                        controlledAsset=id_entity,
                        dateLastValueReported=row["Date"],
                    )
                    
                    if id_entity not in link_piezometro_sensores:
                        link_piezometro_sensores[id_entity] = {}
                    link_piezometro_sensores[id_entity][valorSensor] = id_sensor,
                    crear_subscription(
                        description="Subscription to get info about " + id_entity + " sensor",
                        entity_type=maper_property_sensor[sensores[contador]],
                        watched_attributes=[maper_property_controlled[valorSensor]],
                        sent_attributes=[maper_property_controlled[valorSensor], "controlledAsset", "measureType"],
                        # TODO Cambiar Id por identifier aqui y en el modelo NGSI-LD y en el required
                        query="source==%22"+diccionario_info_piezometros[id_piezometro]["source"]+"%22;controlledProperty==%22"+maper_property_controlled[valorSensor]+"%22",
                        url="http://node-app:3000/historic/"+id_sensor
                    )
                    counters[valorSensor]+=1
            entities_id[id_entity] = diccionario_info_piezometros[id_piezometro]["location"]
            crear_piezometro(
                id=number_entity,
                locality=diccionario_info_piezometros[id_piezometro]["locality"],
                postalCode=diccionario_info_piezometros[id_piezometro]["postalCode"],
                location=diccionario_info_piezometros[id_piezometro]["location"],
                description="Piezometric Point "+ id_piezometro +" which belong to the piezometric net in Mar Menor",
                source=diccionario_info_piezometros[id_piezometro]["source"],
                dateLastValueReported=row["Date"],
                numberInNetwork=id_piezometro.split("Z")[1],
                temperature=check_exists(row,"Temperatura"),
                obsTemp=row["Date"],
                providedByTemp=check_exists(link_piezometro_sensores[id_entity],"Temperatura",array=True),
                conductivity=check_exists(row,"Conductividad"),
                obsCond=row["Date"],
                providedByCond=check_exists(link_piezometro_sensores[id_entity],"Conductividad",array=True),
                piezometricLevel=check_exists(row,"Piezometrico"),
                obsPiez=row["Date"],
                providedByPiez=check_exists(link_piezometro_sensores[id_entity],"Piezometrico",array=True),
                salinity=check_exists(row,"Salinidad"),
                obsSal=row["Date"],
                providedBySal=check_exists(link_piezometro_sensores[id_entity],"Salinidad",array=True),
                tds=check_exists(row,"Total solido disuelto (TDS)"),
                obsTDS=row["Date"],
                providedByTDS=check_exists(link_piezometro_sensores[id_entity],"Total solido disuelto (TDS)",array=True),
                name="Piezometric Point "+id_piezometro,
            )
            num_piezometros += 1
        else:
            for contador,valorSensor in enumerate(sensores):
                if valorSensor == "Date":
                    continue
                id_sensor = f"urn:ngsi-ld:{maper_property_sensor[sensores[contador]]}:{str((counters[valorSensor]-1)).zfill(3)}"
                if not math.isnan(row[valorSensor]):
                    print(id_sensor)
                    print(index)
                    print(row["Date"])
                    modify_entity(
                        value=row[valorSensor],
                        observedAt=row["Date"],
                        url="http://localhost:1026/ngsi-ld/v1/entities/"+id_sensor+"/attrs/"+maper_property_controlled[valorSensor]
                    )
                if(index == len(valorCarpeta)-1 and (not math.isnan(row[valorSensor]))):
                    modify_entity(
                        value=row["Date"],
                        observedAt=None,
                        url="http://localhost:1026/ngsi-ld/v1/entities/"+id_entity+"/attrs/dateLastValueReported"
                    )
                    modify_entity(
                        value=row[valorSensor],
                        observedAt=row["Date"],
                        url="http://localhost:1026/ngsi-ld/v1/entities/"+id_entity+"/attrs/"+maper_property_controlled[valorSensor]
                    )
                if((counter_connections % 2900 == 0) and (contador == 0 or contador == 1)):
                    time.sleep(70)
              
    


{'description': 'Subscription to get info about urn:ngsi-ld:PiezometricPoint:000 sensor', 'type': 'Subscription', 'entities': [{'type': 'TemperatureSensor'}], 'watchedAttributes': ['temperature'], 'notification': {'attributes': ['temperature', 'controlledAsset', 'measureType'], 'format': 'normalized', 'endpoint': {'uri': 'http://node-app:3000/historic/urn:ngsi-ld:TemperatureSensor:000', 'accept': 'application/json'}}, '@context': 'https://raw.githubusercontent.com/mariete1223/MarMenor/main/data_models_description/datamodels.context-ngsi.jsonld', 'q': 'source==%22https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z01E10%22;controlledProperty==%22temperature%22'}
{'description': 'Subscription to get info about urn:ngsi-ld:PiezometricPoint:000 sensor', 'type': 'Subscription', 'entities': [{'type': 'ConductivitySensor'}], 'watchedAttributes': ['conductivity'], 'notification': {'attributes': ['conductivity', 'controlledAsset', 'measureType'], 'format': 'normalized', 'endpoin

In [76]:
diccionario_info_boyas = {
    "E1": {
        "locality": "Mar Menor, San Javier, Cartagena",
        "postalCode": "30380",
        "location": [37.81, -0.78],
        "unitCode": "MTR",
        "source": "https://idearm.imida.es/cgi/siomctdmarmenor/#CTD-E1",
    },
    "E2": {
        "locality": "Mar Menor, Playa de la Hita, Cartagena",
        "postalCode": "30380",
        "location": [37.76, -0.8],
        "unitCode": "MTR",
        "source": "https://idearm.imida.es/cgi/siomctdmarmenor/#CTD-E2",
    },
    "E3": {
        "locality": "Mar Menor, Playa de la Hita, Cartagena",
        "postalCode": "30380",
        "location": [37.76, -0.78],
        "unitCode": "MTR",
        "source": "https://idearm.imida.es/cgi/siomctdmarmenor/#CTD-E3",
    },
    "E4": {
        "locality": "Mar Menor, La Manga, Cartagena",
        "postalCode": "30380",
        "location": [37.74, -0.74],
        "unitCode": "MTR",
        "source": "https://idearm.imida.es/cgi/siomctdmarmenor/#CTD-E4",
    },
    "E5": {
        "locality": "Mar Menor, La Manga, Cartagena",
        "postalCode": "30380",
        "location": [37.74, -0.72],
        "unitCode": "MTR",
        "source": "https://idearm.imida.es/cgi/siomctdmarmenor/#CTD-E5",
    },
    "E6": {
        "locality": "Mar Menor, Isla Mayor, Cartagena",
        "postalCode": "30720",
        "location": [37.71, -0.77],
        "unitCode": "MTR",
        "source": "https://idearm.imida.es/cgi/siomctdmarmenor/#CTD-E6",
    },
    "E7": {
        "locality": "Mar Menor, Los Alcazares, Cartagena",
        "postalCode": "30710",
        "location": [37.71, -0.83],
        "unitCode": "MTR",
        "source": "https://idearm.imida.es/cgi/siomctdmarmenor/#CTD-E7",
    },
    "E8": {
        "locality": "Mar Menor, Los Urrutias, Cartagena",
        "postalCode": "30368",
        "location": [37.69, -0.81],
        "unitCode": "MTR",
        "source": "https://idearm.imida.es/cgi/siomctdmarmenor/#CTD-E8",
    },
    "E9": {
        "locality": "Mar Menor, Saladar de Lo Poyo, Cartagena",
        "postalCode": "30368",
        "location": [37.66, -0.8],
        "unitCode": "MTR",
        "source": "https://idearm.imida.es/cgi/siomctdmarmenor/#CTD-E9",
    },
    "E10": {
        "locality": "Mar Menor, Los Nietos, Cartagena",
        "postalCode": "30383",
        "location": [37.65, -0.78],
        "unitCode": "MTR",
        "source": "https://idearm.imida.es/cgi/siomctdmarmenor/#CTD-E10",
    },
    "E11": {
        "locality": "Mar Menor, La Manga, Cartagena",
        "postalCode": "30380",
        "location": [37.65, -0.72],
        "unitCode": "MTR",
        "source": "https://idearm.imida.es/cgi/siomctdmarmenor/#CTD-E11",
    },
    "E12": {
        "locality": "Mar Menor, Cartagena",
        "postalCode": "30380",
        "location": [37.68, -0.78],
        "unitCode": "MTR",
        "source": "https://idearm.imida.es/cgi/siomctdmarmenor/#CTD-E12",
    },
}

In [77]:
import os
import glob
import pandas as pd
import time

# Especifica la ruta de la carpeta principal
carpeta = "C:\\Users\\marie\\OneDrive\\Escritorio\\Antiguo ordenador\\Universidad\\Master 1º\\TFM\\NGSI-LD\\datos\\BoyasProf\\"

# Utiliza os.listdir() para obtener una lista de elementos en la carpeta
elementos_en_carpeta = os.listdir(carpeta)

# Filtra solo las carpetas en la lista de elementos
carpetas = [elemento for elemento in elementos_en_carpeta if os.path.isdir(os.path.join(carpeta, elemento))]

# DataFrame vacío para almacenar los datos
dfs = {}

time.sleep(60)
# Imprime la lista de carpetas
for carpeta_actual in carpetas:
    print(carpeta_actual)
    df_total = pd.DataFrame()
    archivos_csv = glob.glob(os.path.join(carpeta+carpeta_actual, "*.csv"))
    for archivo in archivos_csv:
        
        nombre_archivo = archivo.split("\\")[-1].split(".")[0]
        df_temporal = pd.read_csv(archivo)
        df_temporal.columns = [df_temporal.columns[0]] + [f"{nombre_archivo}_{col.split('_')[1]}" for col in df_temporal.columns[1:]]
        df_temporal = df_temporal.loc[:, ~df_temporal.columns.str.contains("None")]
        
        if(len(df_total) == 0):
            df_total = df_temporal
        else:
            #Hacer un merge de df_total y df_temporal por la columna "Date"
            df_total = pd.merge(df_total, df_temporal, on="fecha", how="inner")
    dfs[carpeta_actual] = df_total

for key, df in dfs.items():
    # Convierte la columna "Date" a tipo datetime
    df["fecha"] = pd.to_datetime(df["fecha"])
    # Formatea la columna "Date" en el nuevo formato
    df["fecha"] = df["fecha"].dt.strftime("%Y-%m-%dT%H:%M:%SZ")

CTD-E1
CTD-E10
CTD-E11
CTD-E12
CTD-E2
CTD-E3
CTD-E4
CTD-E5
CTD-E6
CTD-E7
CTD-E8
CTD-E9


In [78]:
import math
import time

def formatMultipleValues(values, depths, unitcode, observedAt, providedBy):
    if len(values) == 0:
        return None
    else:
        result = []
        for i in range(len(values)):
            json_data = {
                "measure": {
                    "value": values[i],
                    "unitCode": unitcode
                },
                "depth": {
                    "value": float(depths[i]),
                    "unitCode": "MTR"
                },
                "providedBy": providedBy,
                "observedAt": observedAt
            }
            if providedBy is None:
                del json_data["providedBy"]
            if unitcode is None:
                del json_data["measure"]["unitCode"]
            result.append(json_data)
        return result

maper_property_sensor = { "Clorofila":"ChlorophyllSensor","PH":"PhSensor","Polietileno":"PolyethyleneSensor","Turbidez":"TurbiditySensor","Salinidad":"SalinitySensor",
        "Transparencia":"TransparencySensor","Oxigeno":"OxygenSensor","Temperatura": "TemperatureSensor", "Conductividad": "ConductivitySensor", "Piezometrico":"PiezometricLevelSensor", 
        "Total solido disuelto (TDS)": "TdsSensor", "Materia Organiza":"OrganicMatterSensor"
}

maper_property_controlled = { "Temperatura": "temperature","Materia Organiza":"OrganicMatterSensor", "Conductividad": "conductivity", "Piezometrico":"piezometricLevel", "Salinidad":"salinity",
       "Total solido disuelto (TDS)": "tds", "Clorofila":"chlorophyll","PH":"pH","Polietileno":"polyethylene","Turbidez":"turbidity","Transparencia":"transparency","Oxigeno":"oxygen"
}

maper_property_unitcode = { "Temperatura": "CEL", "Conductividad": "microseconds/cm", "Piezometrico":"msnm", "Salinidad":"PSU",
       "Total solido disuelto (TDS)": "mg/l", "Materia Organiza":None,"Clorofila":None,"PH":"pH",
       "Polietileno":None,"Turbidez":"NTU","Transparencia":None,"Oxigeno":None
}

link_piezometro_sensores = {}

num_boyas = 0

counter_connections = 0

for carpeta, valorCarpeta in dfs.items():
    print(carpeta)
    sensores = list(set( val.split("_")[0] for val in valorCarpeta.columns))
    for i in sensores:
        if counters.get(i) is None:
            counters[i] = 0
    
    id_boya = carpeta.split("-")[1]
    number_entity = str(id_boya[1:]).zfill(3)
    id_entity = f"urn:ngsi-ld:Buoy:{number_entity}"
    for index, row in valorCarpeta.iterrows():
        counter_connections = counter_connections + 1
        if index == 0:
            measuredValues = {}
            for contador,valorSensor in enumerate(sensores):
                if valorSensor == "fecha":
                    continue
                id_sensor = f"urn:ngsi-ld:{maper_property_sensor[sensores[contador]]}:{str(counters[valorSensor]).zfill(3)}"
                try:
                    profundidades, valores = zip(*((col.split("_")[1], row[col]) for col in valorCarpeta.columns if col.startswith(valorSensor) and not pd.isna(row[col]) ))
                except ValueError:
                    profundidades = []
                    valores = []
                profundidades = list(profundidades)
                valores = list(valores)
                measuredValues[valorSensor] = formatMultipleValues(valores,profundidades, maper_property_unitcode[valorSensor], row["fecha"], None)
                crear_sensor(
                     id=str(counters[valorSensor]).zfill(3),
                     sensor_type=maper_property_sensor[sensores[contador]],
                     locality=diccionario_info_boyas[id_boya]["locality"],
                     postalCode=diccionario_info_boyas[id_boya]["postalCode"],
                     location=diccionario_info_boyas[id_boya]["location"],
                     controlledProperty=maper_property_controlled[valorSensor],
                     measuredProperty=maper_property_controlled[valorSensor],
                     measureType="multipleValue",
                     measureValue=measuredValues[valorSensor],
                     unitCode=maper_property_unitcode[valorSensor],
                     source=diccionario_info_boyas[id_boya]["source"],
                     controlledAsset=id_entity,
                     dateLastValueReported=row["fecha"],
                )
                    
                if id_entity not in link_piezometro_sensores:
                    link_piezometro_sensores[id_entity] = {}
                link_piezometro_sensores[id_entity][valorSensor] = id_sensor,
                crear_subscription(
                    description="Subscription to get info about " + id_entity + " sensor",
                    entity_type=maper_property_sensor[sensores[contador]],
                    watched_attributes=[maper_property_controlled[valorSensor]+"Values"],
                    sent_attributes=[maper_property_controlled[valorSensor]+"Values", "controlledAsset", "measureType"],
                    # TODO Cambiar Id por identifier aqui y en el modelo NGSI-LD y en el required
                    query="source==%22"+diccionario_info_boyas[id_boya]["source"]+"%22;controlledProperty==%22"+maper_property_controlled[valorSensor]+"%22",
                    url="http://node-app:3000/historic/"+id_sensor
                )
                counters[valorSensor]+=1
                
            # Añadimos el providedBy
            
            for key, json_object in measuredValues.items():
                if json_object is not None:
                    for value in json_object:
                        value.update({"providedBy": link_piezometro_sensores[id_entity][key][0]})
            # Cambiar a crear Boya aqui
            entities_id[id_entity] = diccionario_info_boyas[id_boya]["location"]
            crear_boya(
                id=number_entity,
                dateLastValueReported=row["fecha"],
                description="Buoy "+ id_boya +" for measuring water quality in the Mar Menor",
                postalCode=diccionario_info_boyas[id_boya]["postalCode"],
                location=diccionario_info_boyas[id_boya]["location"],
                locality=diccionario_info_boyas[id_boya]["locality"],
                name="Buoy " + number_entity,
                source=diccionario_info_boyas[id_boya]["source"],
                temperatureValues=measuredValues["Temperatura"], 
                chlorophyllValues=measuredValues["Clorofila"], 
                conductivityValues=measuredValues["Conductividad"], 
                organicMatterValues=measuredValues["Materia Organiza"],
                oxygenValues=measuredValues["Oxigeno"], 
                pHValues=measuredValues["PH"], 
                polyethyleneValues=measuredValues["Polietileno"], 
                salinityValues=measuredValues["Salinidad"], 
                transparencyValues=measuredValues["Transparencia"], 
                turbidityValues=measuredValues["Turbidez"], 
            )
            num_boyas += 1
        else:
            for contador,valorSensor in enumerate(sensores):
                if valorSensor == "fecha":
                    continue
                id_sensor = f"urn:ngsi-ld:{maper_property_sensor[sensores[contador]]}:{str((counters[valorSensor]-1)).zfill(3)}"
                try:
                    profundidades, valores = zip(*((col.split("_")[1], row[col]) for col in valorCarpeta.columns if col.startswith(valorSensor) and not pd.isna(row[col]) ))
                except ValueError:
                    profundidades = []
                    valores = []
                profundidades = list(profundidades)
                valores = list(valores)
                value_to_patch = formatMultipleValues(valores,profundidades, maper_property_unitcode[valorSensor], row["fecha"], None)
                if value_to_patch is not None:
                    modify_entity(
                        multipleValue=value_to_patch,
                        value=None,
                        observedAt=None,
                        url="http://localhost:1026/ngsi-ld/v1/entities/"+id_sensor+"/attrs/"+maper_property_controlled[valorSensor]+"Values",
                    )    
                if value_to_patch is not None: 
                    for value in value_to_patch:
                        value.update({"providedBy": link_piezometro_sensores[id_entity][valorSensor][0]})
                if(index == len(valorCarpeta)-1):
                    if (value_to_patch is not None):
                        print(value_to_patch)
                        modify_entity(
                            value=row["fecha"],
                            observedAt=None,
                            url="http://localhost:1026/ngsi-ld/v1/entities/"+id_entity+"/attrs/dateLastValueReported"
                        )
                        modify_entity(
                            multipleValue=value_to_patch,
                            value=None,
                            observedAt=None,
                            url="http://localhost:1026/ngsi-ld/v1/entities/"+id_entity+"/attrs/"+maper_property_controlled[valorSensor]+"Values",
                        )
                    else:
                        print(value_to_patch)
                        print("No se ha modificado "+id_sensor+ "de la boya "+id_entity)
                if((counter_connections % 1000 == 0) and (contador == 0 or contador == 1)):
                    time.sleep(80)
              
    

CTD-E1
{'description': 'Subscription to get info about urn:ngsi-ld:Buoy:001 sensor', 'type': 'Subscription', 'entities': [{'type': 'TemperatureSensor'}], 'watchedAttributes': ['temperatureValues'], 'notification': {'attributes': ['temperatureValues', 'controlledAsset', 'measureType'], 'format': 'normalized', 'endpoint': {'uri': 'http://node-app:3000/historic/urn:ngsi-ld:TemperatureSensor:018', 'accept': 'application/json'}}, '@context': 'https://raw.githubusercontent.com/mariete1223/MarMenor/main/data_models_description/datamodels.context-ngsi.jsonld', 'q': 'source==%22https://idearm.imida.es/cgi/siomctdmarmenor/#CTD-E1%22;controlledProperty==%22temperature%22'}
{'description': 'Subscription to get info about urn:ngsi-ld:Buoy:001 sensor', 'type': 'Subscription', 'entities': [{'type': 'SalinitySensor'}], 'watchedAttributes': ['salinityValues'], 'notification': {'attributes': ['salinityValues', 'controlledAsset', 'measureType'], 'format': 'normalized', 'endpoint': {'uri': 'http://node-ap

In [79]:
maper_rambla_id = {
    "06A01": "urn:ngsi-ld:Ravine:001",
    "06A02": "urn:ngsi-ld:Ravine:002",
    "06A03": "urn:ngsi-ld:Ravine:003",
    "06A04": "urn:ngsi-ld:Ravine:004",
    "06A05": "urn:ngsi-ld:Ravine:005",
    "06A06": "urn:ngsi-ld:Ravine:006",
    "06A18": "urn:ngsi-ld:Ravine:007",
    "06A19": "urn:ngsi-ld:Ravine:008",
    "06P02": "urn:ngsi-ld:Ravine:009",
    "06P04": "urn:ngsi-ld:Ravine:010",
    "06P05": "urn:ngsi-ld:Ravine:011",
    "01M02": "urn:ngsi-ld:Ravine:012",
    "06B01": "urn:ngsi-ld:Ravine:013"
}

diccionario_info_ramblas = {
    "06A01": {
        "description": "Marco de control 06A01 en la puebla , rambla del Albujón",
        "locality": "RM-F35, La Puebla, Cartagena",
        "postalCode": "30395",
        "location": [37.7207, -0.9144],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06A01U12",
        "section": 5,
        "nextSection": maper_rambla_id["06A18"],
        "previousSection": maper_rambla_id["06A02"],
    },
    "06A02": {
        "description": "Marco de control 06A02 en Pozo Estrecho, rambla del Albujón",
        "locality": "Pozo Estrecho, Cartagena",
        "postalCode": "30594",
        "location": [37.7239, -0.9793],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06A02U12",
        "section": 4,
        "nextSection": maper_rambla_id["06A01"],
        "previousSection": maper_rambla_id["06A03"],
    },
    "06A03": {
        "description": "Marco de control 06A03 rambla del Albujón",
        "locality": "Rambla de El Albujón, Cartagena",
        "postalCode": "30330",
        "location": [37.7213, -1.0491],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06A03U12",
        "section": 3,
        "nextSection": maper_rambla_id["06A02"],
        "previousSection": maper_rambla_id["06A04"],
    },
    "06A04": {
        "description": "Marco de control 06A04 en el Estrecho, rambla del Albujón",
        "locality": "Estrecho de Fuente Álamo, Cartagena",
        "postalCode": "30332",
        "location": [37.7259, -1.1098],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06A04U12",
        "section": 2,
        "nextSection": maper_rambla_id["06A03"],
        "previousSection": maper_rambla_id["06A05"]+","+maper_rambla_id["06A06"],
    },
    "06A05": {
        "description": "Marco de control 06A05 en Fuente Álamo, rambla del Albujón",
        "locality": "Fuente Álamo, Murcia",
        "postalCode": "30320",
        "location": [37.7247, -1.1721],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06A05U12",
        "section": 1,
        "nextSection": maper_rambla_id["06A04"],
        "previousSection": None,
    },
    "06A06": {
        "description": "Marco de control 06A06 en Los Cegarras, rambla de la Murta",
        "locality": "Cabecico del Rey, Murcia",
        "postalCode": "30154",
        "location": [37.7541, -1.1443],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06A06U12",
        "section": 1,
        "nextSection": maper_rambla_id["06A04"],
        "previousSection": None,
    },
    "06A18": {
        "description": "Desembocadura 06A05 Rambla del Albujón",
        "locality": "Bahía Bella, Cartagena",
        "postalCode": "30368",
        "location": [37.7163, -0.8610],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06A18U12",
        "section": 6,
        "nextSection": None,
        "previousSection": maper_rambla_id["06A01"],
    },
    "06A19": {
        "description": "Marco de control en Rambla de la Maraña- Balsicas",
        "locality": "Los Camachos, Cartagena",
        "postalCode": "30592",
        "location": [37.7953, -0.9415],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06A19U12",
        "section": 1,
        "nextSection": None,
        "previousSection": None,
    },
    "06P02": {
        "description": "Marco de control 06P02 en Rambla de la Maraña- Balsicas",
        "locality": "Torre Pacheco, Murcia",
        "postalCode": "30708",
        "location": [37.7492, -0.9921],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06P02P01",
        "section": 1,
        "nextSection": None,
        "previousSection": None,
    },
    "06P04": {
        "description": "Marco de control 06P04 en Torre Pacheco",
        "locality": "La Murta-Carrascoy, Murcia",
        "postalCode": "30153",
        "location": [37.8270, -1.2107],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06P04P01",
        "section": 1,
        "nextSection": None,
        "previousSection": None,
    },
    "06P05": {
        "description": "Marco de control 06P04 en San Javier (El Mirador)",
        "locality": "El Mirador, San Javier, Murcia",
        "postalCode": "30592",
        "location": [37.8359, -0.8757],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06P05P01",
        "section": 1,
        "nextSection": None,
        "previousSection": None,
    },
    "01M02": {
        "description": "Estación Meteoro-Hidrológica 01M02 el Relojero (AMETSE)",
        "locality": "El Relojero, La Alberca, Murcia",
        "postalCode": "30155",
        "location": [37.9136, -1.1190],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=01M02P01",
        "section": 1,
        "nextSection": None,
        "previousSection": None,
    },
    "06B01": {
        "description": "Elevación 06B01 el Albujón",
        "locality": "Bahía bellas, Los Alcazares, Cartagena",
        "postalCode": "30368",
        "location": [37.7175, -0.8598],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06B01L03",
        "section": 1,
        "nextSection": None,
        "previousSection": None,
    }
}

In [80]:
import os
import glob
import pandas as pd

# Especifica la ruta de la carpeta principal
carpeta = "C:\\Users\\marie\\OneDrive\\Escritorio\\Antiguo ordenador\\Universidad\\Master 1º\\TFM\\NGSI-LD\\datos\\SAIHdatasActualizados\\Ramblas\\"

# Utiliza os.listdir() para obtener una lista de elementos en la carpeta
elementos_en_carpeta = os.listdir(carpeta)

# Filtra solo las carpetas en la lista de elementos
carpetas = [elemento for elemento in elementos_en_carpeta if os.path.isdir(os.path.join(carpeta, elemento))]

# DataFrame vacío para almacenar los datos
dfs = {}


# Imprime la lista de carpetas
for carpeta_actual in carpetas:
    df_total = pd.DataFrame()
    archivos_csv = glob.glob(os.path.join(carpeta+carpeta_actual, "*.csv"))
    for archivo in archivos_csv:
    
        df_temporal = pd.read_csv(archivo)
        if(len(df_total) == 0):
            df_total = df_temporal
        else:
            #Hacer un merge de df_total y df_temporal por la columna "Date"
            df_total = pd.merge(df_total, df_temporal, on="Date", how="inner")
    dfs[carpeta_actual] = df_total

for key, df in dfs.items():
    # Convierte la columna "Date" a tipo datetime
    df["Date"] = pd.to_datetime(df["Date"])
    # Formatea la columna "Date" en el nuevo formato
    df["Date"] = df["Date"].dt.strftime("%Y-%m-%dT%H:%M:%SZ")

In [81]:
import math
import time

def formatMultipleValues(values, unitcode, observedAt, providedBy):
    if len(values) == 0:
        return None
    else:
        result = []
        for i in range(len(values)):
            json_data = {
                "measure": {
                    "value": values[i],
                    "unitCode": unitcode
                },
                "providedBy": providedBy,
                "observedAt": observedAt
            }
            if providedBy is None:
                del json_data["providedBy"]
            if unitcode is None:
                del json_data["measure"]["unitCode"]
            result.append(json_data)
        return result
    
def check_exists(place_to_search, value_to_search, array=False):
    if array:
        return place_to_search[value_to_search][0] if value_to_search in place_to_search else None

    return place_to_search[value_to_search] if value_to_search in place_to_search else None

maper_property_sensor = { "Volumen":"VolumeSensor","Presion":"PressureSensor","Temperatura": "TemperatureSensor", "Presion Atmosferica":"AtmosphericPressureSensor","Direccion del Viento":"WindDirectionSensor","Velocidad del Viento":"WindSpeedSensor" ,"Nivel": "WaterLevelSensor", "Pluviometro":"PrecipitationSensor","Caudal":"WaterFlowSensor", "Humedad":"HumiditySensor","Conductividad":"ConductivitySensor",
}

maper_property_controlled = { "Volumen":"volume","Presion":"pressure","Presion Atmosferica":"atmosphericPressure","Direccion del Viento":"windDirection","Velocidad del Viento":"windSpeed","Temperatura": "temperature", "Nivel": "waterLevel", "Pluviometro":"precipitation","Caudal":"waterFlow", "Humedad":"humidity","Conductividad":"conductivity"
}

maper_property_unitcode = { "Volumen":"m3","Presion":"bar", "Presion Atmosferica":"mbar", "Direccion del Viento":"°","Velocidad del Viento":"m/s","Temperatura": "CEL", "Conductividad": "microseconds/cm", "Nivel": "m", "Pluviometro":"mm","Caudal":"m3/s", "Humedad":"%"
}

link_rambla_sensores = {}

num_ramblas = 0

counter_connections = 0
counters = {}

for carpeta, valorCarpeta in dfs.items():
    print(carpeta)
    sensores = list(set( val.split("_")[0] for val in valorCarpeta.columns))
    for i in sensores:
        if counters.get(i) is None:
            counters[i] = 0
    print(sensores)
    id_rambla = carpeta.split("-")[0]
    if (id_rambla not in diccionario_info_ramblas):
        continue
    
    id_entity = maper_rambla_id[id_rambla]
    number_entity = id_entity.split(":")[-1]
    for index, row in valorCarpeta.iterrows():
        waterLevel_values = []
        waterFlow_values = []
        counter_connections = counter_connections + 1
        if index == 0:
            measurement_places = {}
            for contador,valorSensor in enumerate(sensores):
                original = valorSensor
                if valorSensor == "Date":
                    continue
                if len(valorSensor.split(" "))>0 and valorSensor not in maper_property_controlled:
                    measurementPlace = " ".join(valorSensor.split(" ")[1:])
                    valorSensor = valorSensor.split(" ")[0]
                    measurement_places[maper_property_controlled[valorSensor]] = measurementPlace
                    if valorSensor not in counters:
                        counters[valorSensor] = 0

                id_sensor = f"urn:ngsi-ld:{maper_property_sensor[valorSensor]}:{str(counters[valorSensor]).zfill(3)}"

                if "Nivel" in valorSensor or "Caudal" in valorSensor:
                    measure_location = None
                    if len(original.split(" "))>0:
                        measure_location = " ".join(original.split(" ")[1:])
                    nuevo_valor = {"measure": {"value": row[original],"unitCode": maper_property_unitcode[valorSensor]}, "place": {"type":"property","value":measure_location},"providedBy": id_sensor,"observedAt": row["Date"]}
                    if measure_location is None:
                        del nuevo_valor["place"]
                    if "Nivel" in valorSensor:
                        waterLevel_values.append(nuevo_valor)
                    else:
                        waterFlow_values.append(nuevo_valor)
                    continue
                crear_sensor(
                     id=str(counters[valorSensor]).zfill(3),
                     sensor_type=maper_property_sensor[valorSensor],
                     locality=diccionario_info_ramblas[id_rambla]["locality"],
                     postalCode=diccionario_info_ramblas[id_rambla]["postalCode"],
                     location=diccionario_info_ramblas[id_rambla]["location"],
                     controlledProperty=maper_property_controlled[valorSensor],
                     measuredProperty=maper_property_controlled[valorSensor],
                     measureType="multipleValue" if valorSensor == "Caudal" or valorSensor == "Nivel" else "singleValue",
                     measureValue=formatMultipleValues([row[original]],maper_property_unitcode[valorSensor], row["Date"], None) if valorSensor == "Caudal" or valorSensor == "Nivel" else row[original],
                     unitCode=maper_property_unitcode[valorSensor],
                     source=diccionario_info_ramblas[id_rambla]["source"],
                     controlledAsset=id_entity,
                     dateLastValueReported=row["Date"],
                )
                    
                if id_entity not in link_rambla_sensores:
                    link_rambla_sensores[id_entity] = {}
                link_rambla_sensores[id_entity][valorSensor] = id_sensor,
                watched_attributes_value = maper_property_controlled[valorSensor]+"Values" if valorSensor == "Caudal" or valorSensor == "Nivel" else  maper_property_controlled[valorSensor]
                crear_subscription(
                    description="Subscription to get info about " + id_entity + " sensor",
                    entity_type=maper_property_sensor[valorSensor],
                    watched_attributes=[watched_attributes_value],
                    sent_attributes=[watched_attributes_value, "controlledAsset", "measureType"],
                    # TODO Cambiar Id por identifier aqui y en el modelo NGSI-LD y en el required
                    query="source==%22"+diccionario_info_ramblas[id_rambla]["source"]+"%22;controlledProperty==%22"+maper_property_controlled[valorSensor]+"%22",
                    url="http://node-app:3000/historic/"+id_sensor
                )
                counters[valorSensor]+=1
            for column_name,values_collection in zip(["Nivel","Caudal"],[waterLevel_values, waterFlow_values]):
                if(len(values_collection) > 0):
                    crear_sensor(
                     id=str(counters[column_name]).zfill(3),
                     sensor_type=maper_property_sensor[column_name],
                     locality=diccionario_info_ramblas[id_rambla]["locality"],
                     postalCode=diccionario_info_ramblas[id_rambla]["postalCode"],
                     location=diccionario_info_ramblas[id_rambla]["location"],
                     controlledProperty=maper_property_controlled[column_name],
                     measuredProperty=maper_property_controlled[column_name],
                     measureType="multipleValue",
                     measureValue= values_collection,
                     unitCode=maper_property_unitcode[column_name],
                     source=diccionario_info_ramblas[id_rambla]["source"],
                     controlledAsset=id_entity,
                     dateLastValueReported=row["Date"],
                    )
                    
                    if id_entity not in link_rambla_sensores:
                        link_rambla_sensores[id_entity] = {}
                    link_rambla_sensores[id_entity][column_name] = id_sensor,
                    watched_attributes_value = maper_property_controlled[column_name]+"Values" if column_name == "Caudal" or column_name == "Nivel" else  maper_property_controlled[column_name]
                    crear_subscription(
                        description="Subscription to get info about " + id_entity + " sensor",
                        entity_type=maper_property_sensor[column_name],
                        watched_attributes=[watched_attributes_value],
                        sent_attributes=[watched_attributes_value, "controlledAsset", "measureType"],
                        # TODO Cambiar Id por identifier aqui y en el modelo NGSI-LD y en el required
                        query="source==%22"+diccionario_info_ramblas[id_rambla]["source"]+"%22;controlledProperty==%22"+maper_property_controlled[column_name]+"%22",
                        url="http://node-app:3000/historic/"+id_sensor
                    )
                    counters[column_name]+=1
            
            fecha = row["Date"]
            entities_id[id_entity] = diccionario_info_ramblas[id_rambla]["location"]
            conductivity = next((cadena for cadena in row.index if "Conductividad" in cadena), None)
            pressure = next((cadena for cadena in row.index if "Presion" in cadena), None) if next((cadena for cadena in row.index if "Presion" in cadena), None) != "Presion Atmosferica" else None
            volume = next((cadena for cadena in row.index if "Volumen" in cadena), None)
            print(conductivity)
            print(pressure)
            print(volume)
            crear_rambla(
                id=number_entity,
                dateLastValueReported=fecha,
                description=diccionario_info_ramblas[id_rambla]["description"],
                locality=diccionario_info_ramblas[id_rambla]["locality"],
                postalCode=diccionario_info_ramblas[id_rambla]["postalCode"],
                source=diccionario_info_ramblas[id_rambla]["source"],
                location=diccionario_info_ramblas[id_rambla]["location"],
                temperature=row["Temperatura"] if "Temperatura" in row else None, 
                providedByTemp= check_exists(link_rambla_sensores[id_entity],"Temperatura",array=True),
                obsTemp=fecha,
                precipitation=row["Pluviometro"] if "Pluviometro" in row else None, 
                providedByPrec= check_exists(link_rambla_sensores[id_entity],"Pluviometro",array=True),
                obsPrec=fecha,
                humidity=row["Humedad"] if "Humedad" in row else None, 
                providedByHum= check_exists(link_rambla_sensores[id_entity],"Humedad",array=True),
                obsHum=fecha,
                atmosphericPressure=row["Presion Atmosferica"] if "Presion Atmosferica" in row else None,
                providedByAtPress= check_exists(link_rambla_sensores[id_entity],"Presion Atmosferica",array=True),
                obsAtPress=fecha,
                windSpeed=row["Velocidad del Viento"] if "Velocidad del Viento" in row else None,
                providedBywindSp= check_exists(link_rambla_sensores[id_entity],"Velocidad del Viento",array=True),
                obsWindSp=fecha,
                windDirection=row["Direccion del Viento"] if "Direccion del Viento" in row else None,
                providedByWindDir= check_exists(link_rambla_sensores[id_entity],"Direccion del Viento",array=True),
                obsWindDir=fecha,
                conductivity= row[conductivity] if conductivity != None else None,
                providedByCond= check_exists(link_rambla_sensores[id_entity],conductivity,array=True),
                obsCond= fecha,
                volume= row[volume] if volume != None else None,
                providedByVol= check_exists(link_rambla_sensores[id_entity],volume,array=True),
                obsVol= fecha,
                pressure= row[pressure] if pressure != None else None,
                providedByPress= check_exists(link_rambla_sensores[id_entity],pressure,array=True),
                obsPress= fecha,
                waterFlowValues=[waterFlow_values] if any("Caudal" in cadena for cadena in row.index) else None, 
                waterLevelValues=[waterLevel_values] if any("Nivel" in cadena for cadena in row.index) else None, 
                section=diccionario_info_ramblas[id_rambla]["section"],
                nextSection=diccionario_info_ramblas[id_rambla]["nextSection"],
                measurementsPlaces=measurement_places,
                previousSection=diccionario_info_ramblas[id_rambla]["previousSection"],
                name="Ravine " + id_rambla
            )
            num_ramblas += 1
        else:
            waterLevel_values = []
            waterFlow_values = []
            for contador,valorSensor in enumerate(sensores):    
                original = valorSensor
                if valorSensor == "Date":
                    continue
                if len(valorSensor.split(" "))>0 and valorSensor not in maper_property_controlled:
                    measurementPlace = " ".join(valorSensor.split(" ")[1:])
                    valorSensor = valorSensor.split(" ")[0]
                    measurement_places[maper_property_controlled[valorSensor]] = measurementPlace
                    if valorSensor not in counters:
                        counters[valorSensor] = 0

                id_sensor = f"urn:ngsi-ld:{maper_property_sensor[valorSensor]}:{str((counters[valorSensor]-1)).zfill(3)}"

                if "Nivel" in valorSensor or "Caudal" in valorSensor:
                    measure_location = None
                    if len(original.split(" "))>0:
                        measure_location = " ".join(original.split(" ")[1:])
                    nuevo_valor = {"measure": {"value": row[original],"unitCode": maper_property_unitcode[valorSensor]}, "place": {"type":"property","value":measure_location},"providedBy": id_sensor,"observedAt": row["Date"]}
                    if measure_location is None:
                        del nuevo_valor["place"]
                    if "Nivel" in valorSensor:
                        waterLevel_values.append(nuevo_valor)
                    else:
                        waterFlow_values.append(nuevo_valor)
                    continue

                if not math.isnan(row[original]):
                    modify_entity(
                        value=row[original],
                        observedAt=row["Date"],
                        url="http://localhost:1026/ngsi-ld/v1/entities/"+id_sensor+"/attrs/"+maper_property_controlled[valorSensor]
                    )
                if(index == len(valorCarpeta)-1 and (not math.isnan(row[original]))):
                    modify_entity(
                        value=row["Date"],
                        observedAt=None,
                        url="http://localhost:1026/ngsi-ld/v1/entities/"+id_entity+"/attrs/dateLastValueReported"
                    )
                    modify_entity(
                        place= measurement_places[maper_property_controlled[valorSensor]] if maper_property_controlled[valorSensor] in measurement_places else None,
                        value=row[original],
                        observedAt=row["Date"],
                        url="http://localhost:1026/ngsi-ld/v1/entities/"+id_entity+"/attrs/"+maper_property_controlled[valorSensor]
                    )

                if((counter_connections % 2900 == 0) and (contador == 0 or contador == 1)):
                    time.sleep(70)
            for column_name,values_collection in zip(["Nivel","Caudal"],[waterLevel_values, waterFlow_values]):
                if(len(values_collection) > 0):
                    modify_entity(
                        multipleValue=[{"measure": {"value": row[original],"unitCode": maper_property_unitcode[valorSensor]},"providedBy": link_rambla_sensores[id_entity][valorSensor],"observedAt": fecha}],
                        value=None,
                        observedAt=None,
                        url="http://localhost:1026/ngsi-ld/v1/entities/"+id_sensor+"/attrs/"+maper_property_controlled[valorSensor]+"Values",
                    )
                    if(index == len(valorCarpeta)-1 and (not math.isnan(row[original]))):
                        modify_entity(
                        value=row["Date"],
                        observedAt=None,
                        url="http://localhost:1026/ngsi-ld/v1/entities/"+id_entity+"/attrs/dateLastValueReported"
                        )
                        modify_entity(
                            multipleValue=[values_collection],
                            value=None,
                            observedAt=None,
                            url="http://localhost:1026/ngsi-ld/v1/entities/"+id_sensor+"/attrs/"+maper_property_controlled[valorSensor]+"Values",
                        )
                    
              

01M02-Relojero
['Temperatura', 'Date', 'Pluviometro', 'Velocidad del Viento', 'Humedad', 'Presion Atmosferica', 'Direccion del Viento']
urn:ngsi-ld:TemperatureSensor:000
{'description': 'Subscription to get info about urn:ngsi-ld:Ravine:012 sensor', 'type': 'Subscription', 'entities': [{'type': 'TemperatureSensor'}], 'watchedAttributes': ['temperature'], 'notification': {'attributes': ['temperature', 'controlledAsset', 'measureType'], 'format': 'normalized', 'endpoint': {'uri': 'http://node-app:3000/historic/urn:ngsi-ld:TemperatureSensor:000', 'accept': 'application/json'}}, '@context': 'https://raw.githubusercontent.com/mariete1223/MarMenor/main/data_models_description/datamodels.context-ngsi.jsonld', 'q': 'source==%22https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=01M02P01%22;controlledProperty==%22temperature%22'}
{'description': 'Subscription to get info about urn:ngsi-ld:Ravine:012 sensor', 'type': 'Subscription', 'entities': [{'type': 'PrecipitationSensor'}], 'wa

In [82]:
for entity_id, location in entities_id.items():
    close_entities = search_close_entities(location[0], location[1],entity_id)
    print(entity_id)
    create_attribute(
        multipleValue=[{"type":"Relationship","object": close_entity} for close_entity in close_entities],
        value=None,
        observedAt=None,
        url="http://localhost:1026/ngsi-ld/v1/entities/"+entity_id+"/attrs",
        newAttributeName="closeMeasurements"
    )

urn:ngsi-ld:PiezometricPoint:000
204 
urn:ngsi-ld:PiezometricPoint:001
204 
urn:ngsi-ld:PiezometricPoint:002
204 
urn:ngsi-ld:PiezometricPoint:003
204 
urn:ngsi-ld:PiezometricPoint:004
204 
urn:ngsi-ld:PiezometricPoint:005
204 
urn:ngsi-ld:PiezometricPoint:006
204 
urn:ngsi-ld:PiezometricPoint:007
204 
urn:ngsi-ld:PiezometricPoint:008
204 
urn:ngsi-ld:PiezometricPoint:009
204 
urn:ngsi-ld:PiezometricPoint:010
204 
urn:ngsi-ld:PiezometricPoint:011
204 
urn:ngsi-ld:PiezometricPoint:012
204 
urn:ngsi-ld:PiezometricPoint:013
204 
urn:ngsi-ld:PiezometricPoint:014
204 
urn:ngsi-ld:PiezometricPoint:015
204 
urn:ngsi-ld:PiezometricPoint:016
204 
urn:ngsi-ld:PiezometricPoint:017
204 
urn:ngsi-ld:PiezometricPoint:018
204 
urn:ngsi-ld:Buoy:001
204 
urn:ngsi-ld:Buoy:010
204 
urn:ngsi-ld:Buoy:011
204 
urn:ngsi-ld:Buoy:012
204 
urn:ngsi-ld:Buoy:002
204 
urn:ngsi-ld:Buoy:003
204 
urn:ngsi-ld:Buoy:004
204 
urn:ngsi-ld:Buoy:005
204 
urn:ngsi-ld:Buoy:006
204 
urn:ngsi-ld:Buoy:007
204 
urn:ngsi-ld:Buoy:0